In [16]:
import ee

# 使用项目ID初始化
try:
    # 尝试初始化
    ee.Initialize(project='gee-satellite-data-456115')
    print("Earth Engine 初始化成功！")
except Exception as e:
    print("初始化失败，需要认证:", str(e))
    # 认证流程
    ee.Authenticate()
    # 认证成功后再次初始化
    ee.Initialize(project='gee-satellite-data-456115')
    print("认证并初始化成功！")

Earth Engine 初始化成功！


In [17]:
import ee
import geemap
import os
import datetime
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

In [18]:
# use local shapefile to define the study area
def get_roi_from_shapefile(shapefile_path):
    roi_asset = geemap.shp_to_ee(shapefile_path)
    if isinstance(roi_asset, ee.FeatureCollection):
        roi_geometry = roi_asset.geometry()
    else:
        roi_geometry = roi_asset
    return roi_geometry

# load the study area boundary
shapefile_path = 'shapefile/Gelderse_Poort.shp'
roi = get_roi_from_shapefile(shapefile_path)

## Get landsat data (1993-2015)

In [19]:
# 1. 获取Landsat数据（1993-2015）
def get_landsat_collection(start_year, end_year):
    """获取Landsat数据集并预处理"""
    
    print(f"获取Landsat数据: {start_year}-{min(end_year, 2015)}")
    
    # Landsat 5 (1993-2011)
    l5 = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2') \
        .filterDate(f'{max(1993, start_year)}-01-01', '2011-12-31') \
        .filterBounds(roi) \
        .filter(ee.Filter.lt('CLOUD_COVER', 20))
    
    # Landsat 7 (1999-2013)
    l7 = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2') \
        .filterDate(f'{max(1999, start_year)}-01-01', '2013-12-31') \
        .filterBounds(roi) \
        .filter(ee.Filter.lt('CLOUD_COVER', 20))
    
    # Landsat 8 (2013-2015)
    l8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
        .filterDate('2013-01-01', f'{min(end_year, 2015)}-12-31') \
        .filterBounds(roi) \
        .filter(ee.Filter.lt('CLOUD_COVER', 20))
    
    # 获取影像数量
    l5_count = l5.size().getInfo()
    l7_count = l7.size().getInfo()
    l8_count = l8.size().getInfo()
    
    print(f"Landsat 5 影像数量: {l5_count}")
    print(f"Landsat 7 影像数量: {l7_count}")
    print(f"Landsat 8 影像数量: {l8_count}")

    # 定义预处理函数 (Landsat 5/7)
    def preprocess_l57(image):
        # 选择地表反射率波段并进行缩放
        sr = image.select(['SR_B3', 'SR_B4']).multiply(0.0000275).add(-0.2)
        # 基于QA波段创建云掩膜
        qa = image.select('QA_PIXEL')
        cloud_mask = qa.bitwiseAnd(1 << 3).eq(0)  # 云位元
        # 计算NDVI: (NIR - RED) / (NIR + RED)
        ndvi = sr.normalizedDifference(['SR_B4', 'SR_B3']).rename('NDVI')
        # 添加时间信息
        date = ee.Date(image.get('system:time_start'))
        year = date.get('year')
        month = date.get('month')
        day = date.get('day')
        
        # 裁剪到研究区域并应用云掩膜
        return image.addBands(ndvi).updateMask(cloud_mask).clip(roi) \
            .set('year', year) \
            .set('month', month) \
            .set('day', day) \
            .set('date', date.format('YYYY-MM-dd'))

    # 定义预处理函数 (Landsat 8)
    def preprocess_l8(image):
        # 选择地表反射率波段并进行缩放
        sr = image.select(['SR_B4', 'SR_B5']).multiply(0.0000275).add(-0.2)
        # 基于QA波段创建云掩膜
        qa = image.select('QA_PIXEL')
        cloud_mask = qa.bitwiseAnd(1 << 3).eq(0)  # 云位元
        # 计算NDVI: (NIR - RED) / (NIR + RED)
        ndvi = sr.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
        # 添加时间信息
        date = ee.Date(image.get('system:time_start'))
        year = date.get('year')
        month = date.get('month')
        day = date.get('day')
        
        # 裁剪到研究区域并应用云掩膜
        return image.addBands(ndvi).updateMask(cloud_mask).clip(roi) \
            .set('year', year) \
            .set('month', month) \
            .set('day', day) \
            .set('date', date.format('YYYY-MM-dd'))

    # 处理每个集合
    print("正在处理Landsat影像...")
    l5_processed = l5.map(preprocess_l57)
    l7_processed = l7.map(preprocess_l57)
    l8_processed = l8.map(preprocess_l8)

    # 合并集合
    merged = ee.ImageCollection(l5_processed.merge(l7_processed).merge(l8_processed))
    total_count = merged.size().getInfo()
    print(f"Landsat数据处理完成，共 {total_count} 景影像")

    return merged

## Get sentinel2 data

In [20]:
# 2. 获取Sentinel-2数据（2015-2023）
def get_sentinel2_collection(start_year, end_year):
    """获取Sentinel-2数据集并预处理"""
    
    # 确保年份在有效范围内 (2015-2023)
    start_year = max(2015, start_year)
    end_year = min(2023, end_year)
    
    print(f"获取Sentinel-2数据: {start_year}-{end_year}")
    
    # 获取Sentinel-2地表反射率数据
    s2 = ee.ImageCollection('COPERNICUS/S2_SR') \
        .filterDate(f'{start_year}-01-01', f'{end_year}-12-31') \
        .filterBounds(roi) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    
    s2_count = s2.size().getInfo()
    print(f"Sentinel-2 影像数量: {s2_count}")
    
    # 预处理函数
    def preprocess_s2(image):
        # 选择需要的波段
        bands = image.select(['B4', 'B8'])
        # 云掩膜 (基于SCL波段)
        scl = image.select('SCL')
        valid_mask = scl.gte(4).And(scl.lte(6))  # 4=植被,5=裸地,6=水
        # 计算NDVI: (NIR - RED) / (NIR + RED)
        ndvi = bands.normalizedDifference(['B8', 'B4']).rename('NDVI')
        # 添加时间信息
        date = ee.Date(image.get('system:time_start'))
        year = date.get('year')
        month = date.get('month')
        day = date.get('day')
        
        # 裁剪到研究区域并应用云掩膜
        return image.addBands(ndvi).updateMask(valid_mask).clip(roi) \
            .set('year', year) \
            .set('month', month) \
            .set('day', day) \
            .set('date', date.format('YYYY-MM-dd'))
    
    print("正在处理Sentinel-2影像...")
    processed = s2.map(preprocess_s2)
    print(f"Sentinel-2数据处理完成，共 {processed.size().getInfo()} 景影像")
    
    return processed

In [21]:
def create_time_composites_fixed(collection, start_year, end_year, temporal_scale='month'):
    """
    创建不同时间尺度的NDVI合成产品
    temporal_scale: 'month', 'season', 'year'
    """
    print(f"创建{temporal_scale}尺度的NDVI合成产品...")
    composites = []
    
    # 获取当前时间作为比较基准
    current_time = ee.Date(datetime.datetime.now().strftime('%Y-%m-%d'))
    
    if temporal_scale == 'month':
        for year in range(start_year, end_year + 1):
            for month in range(1, 13):
                start_date = ee.Date.fromYMD(year, month, 1)
                end_date = start_date.advance(1, 'month')
                
                # 提取当月数据
                filtered = collection.filterDate(start_date, end_date)
                
                # 跳过没有数据的月份
                if filtered.size().getInfo() > 0:
                    # 计算月度中值合成
                    monthly = filtered.select('NDVI') \
                        .median() \
                        .set({
                            'system:time_start': start_date.millis(),
                            'year': year,
                            'month': month,
                            'temporal_scale': 'monthly'
                        })
                    
                    composites.append(monthly)
    
    elif temporal_scale == 'season':
        # 定义季节 (北半球)
        seasons = {
            'spring': [3, 4, 5],     # 春季: 3-5月
            'summer': [6, 7, 8],     # 夏季: 6-8月
            'autumn': [9, 10, 11],   # 秋季: 9-11月
            'winter': [12, 1, 2]     # 冬季: 12-2月(跨年)
        }
        
        for year in range(start_year, end_year + 1):
            for season_name, months in seasons.items():
                if season_name == 'winter':
                    # 冬季跨年处理
                    if year < end_year:
                        start_date = ee.Date.fromYMD(year, 12, 1)
                        end_date = ee.Date.fromYMD(year + 1, 3, 1)
                else:
                    # 其他季节
                    start_date = ee.Date.fromYMD(year, min(months), 1)
                    end_date = ee.Date.fromYMD(year, max(months), 1).advance(1, 'month')
                
                # 跳过未来的季节
                if end_date.millis().getInfo() <= current_time.millis().getInfo():
                    # 提取数据
                    filtered = collection.filterDate(start_date, end_date)
                    
                    # 跳过没有数据的季节
                    if filtered.size().getInfo() > 0:
                        # 季节中值合成
                        seasonal = filtered.select('NDVI') \
                            .median() \
                            .set({
                                'system:time_start': start_date.millis(),
                                'year': year,
                                'season': season_name,
                                'temporal_scale': 'seasonal'
                            })
                        
                        composites.append(seasonal)
    
    elif temporal_scale == 'year':
        for year in range(start_year, end_year + 1):
            # 生长季最大值合成 (5月到9月)
            growing_start = ee.Date.fromYMD(year, 5, 1)
            growing_end = ee.Date.fromYMD(year, 9, 30)
            
            # 跳过未来的年份
            if growing_end.millis().getInfo() <= current_time.millis().getInfo():
                # 提取生长季数据
                filtered = collection.filterDate(growing_start, growing_end)
                
                # 跳过没有数据的年份
                if filtered.size().getInfo() > 0:
                    # 年度最大值合成
                    yearly = filtered.select('NDVI') \
                        .max() \
                        .set({
                            'system:time_start': ee.Date.fromYMD(year, 1, 1).millis(),
                            'year': year,
                            'temporal_scale': 'yearly'
                        })
                    
                    composites.append(yearly)
    
    # 转换为影像集合
    result = ee.ImageCollection.fromImages(composites)
    print(f"完成，共创建 {result.size().getInfo()} 个合成产品")
    return result

In [22]:
# 4. 提取干扰事件数据 (暂时还未定洪水和干旱事件的时间)
def extract_disturbance_events_fixed():
    """提取2018年干旱和洪水事件的NDVI数据（修复版本）"""
    
    print("提取2018年干旱和洪水事件数据...")
    
    # 获取2018年Sentinel-2数据 (10m分辨率)
    s2_2018 = get_sentinel2_collection(2018, 2018)
    
    # 获取2017年底到2018年初的数据 (用于洪水事件)
    s2_winter = ee.ImageCollection(
        get_sentinel2_collection(2017, 2017).merge(
        get_sentinel2_collection(2018, 2018))
    ).filterDate('2017-12-01', '2018-02-28')
    
    print("处理干旱事件数据...")
    # 干旱事件提取 (2018年夏季)
    drought_before = s2_2018.filterDate('2018-05-01', '2018-06-30').select('NDVI').median().clip(roi)
    drought_during = s2_2018.filterDate('2018-07-01', '2018-08-31').select('NDVI').median().clip(roi)
    drought_after = s2_2018.filterDate('2018-09-01', '2018-10-31').select('NDVI').median().clip(roi)
    
    print("处理洪水事件数据...")
    # 洪水事件提取 (2018年1月)
    # 确保选择NDVI波段并处理潜在的空集合
    flood_collection_before = s2_winter.filterDate('2017-12-20', '2018-01-04')
    flood_collection_during = s2_winter.filterDate('2018-01-05', '2018-01-15')
    flood_collection_after = s2_winter.filterDate('2018-01-16', '2018-02-10')
    
    # 检查集合是否为空
    if flood_collection_before.size().getInfo() > 0:
        flood_before = flood_collection_before.select('NDVI').median().clip(roi)
    else:
        print("警告: 洪水前期数据为空，使用空白影像")
        flood_before = ee.Image(0).rename('NDVI').clip(roi)
    
    if flood_collection_during.size().getInfo() > 0:
        flood_during = flood_collection_during.select('NDVI').median().clip(roi)
    else:
        print("警告: 洪水期间数据为空，使用空白影像")
        flood_during = ee.Image(0).rename('NDVI').clip(roi)
    
    if flood_collection_after.size().getInfo() > 0:
        flood_after = flood_collection_after.select('NDVI').median().clip(roi)
    else:
        print("警告: 洪水后期数据为空，使用空白影像")
        flood_after = ee.Image(0).rename('NDVI').clip(roi)
    
    # 设置属性
    drought_before = drought_before.set({'event': 'drought', 'stage': 'before'})
    drought_during = drought_during.set({'event': 'drought', 'stage': 'during'})
    drought_after = drought_after.set({'event': 'drought', 'stage': 'after'})
    
    flood_before = flood_before.set({'event': 'flood', 'stage': 'before'})
    flood_during = flood_during.set({'event': 'flood', 'stage': 'during'})
    flood_after = flood_after.set({'event': 'flood', 'stage': 'after'})
    
    print("干扰事件数据提取完成")
    
    return {
        'drought': {
            'before': drought_before,
            'during': drought_during,
            'after': drought_after
        },
        'flood': {
            'before': flood_before,
            'during': flood_during,
            'after': flood_after
        }
    }

In [45]:
# 5. 数据导出函数
def export_to_drive(image, description, parent_folder, subfolder, scale=30):
    """将影像导出到Google Drive"""
    
    # 获取研究区域边界
    region = roi.bounds().getInfo()['coordinates']
    
    # 确保影像有波段
    if image.bandNames().size().getInfo() == 0:
        print(f"警告: 影像 {description} 没有波段，尝试添加默认波段...")
        image = image.select([0]).rename('NDVI')
    
    # 创建导出任务
    export_task = ee.batch.Export.image.toDrive(
        image=image,
        description=f"{subfolder}_{description}",  # 加上子分类前缀
        folder=parent_folder,  # 只使用父文件夹
        scale=scale,
        region=region,
        maxPixels=1e13,
        fileFormat='GeoTIFF',
        formatOptions={'cloudOptimized': True}
    )
    
    # 启动任务
    export_task.start()
    print(f"导出任务已提交: {subfolder}_{description} → {parent_folder}")
    
    return export_task

In [46]:
def test_download_fixed(start_year=2018, end_year=2020, parent_folder='GEE'):
    """测试下载一小部分数据（修复版本）
    
    Args:
        start_year: 起始年份
        end_year: 结束年份
        parent_folder: Google Drive中的父文件夹名称
    """
    
    print(f"开始测试下载 {start_year}-{end_year} 年的卫星数据...")
    
    # 1. 获取早期数据 (Landsat)
    if start_year < 2015:
        landsat = get_landsat_collection(start_year, min(end_year, 2015))
    else:
        landsat = None
    
    # 2. 获取近期数据 (Sentinel-2)
    if end_year >= 2015:
        sentinel = get_sentinel2_collection(max(start_year, 2015), end_year)
    else:
        sentinel = None
    
    # 3. 合并数据集
    if landsat and sentinel:
        all_data = ee.ImageCollection(landsat.merge(sentinel))
    elif landsat:
        all_data = landsat
    else:
        all_data = sentinel
    
    # 4. 创建年度合成产品示例 (使用修复版本的函数)
    yearly_composites = create_time_composites_fixed(all_data, start_year, end_year, 'year')
    yearly_list = yearly_composites.toList(yearly_composites.size())
    yearly_size = yearly_composites.size().getInfo()
    
    # 5. 导出年度NDVI示例
    print("开始导出年度NDVI示例...")
    for i in range(yearly_size):
        year_img = ee.Image(yearly_list.get(i))
        year = year_img.get('year').getInfo()
        export_to_drive(
            image=year_img,
            description=f'annual_{year}',
            subfolder='NDVI',  # 这将作为文件名前缀
            parent_folder=parent_folder,
            scale=30
    )
    
    # 6. 如果时间范围包含2018年，则提取干扰事件数据
    if start_year <= 2018 <= end_year:
        print("提取2018年干扰事件数据...")
        disturbance_events = extract_disturbance_events_fixed()
        
        # 导出干旱事件示例
        print("导出干旱事件数据示例...")
        for stage, img in disturbance_events['drought'].items():
            export_to_drive(
            image=img,
            description=f'2018_{stage}',
            subfolder='Drought',  
            parent_folder=parent_folder,
            scale=10  
            )
        
        # 导出洪水事件示例
        print("导出洪水事件数据示例...")
        for stage, img in disturbance_events['flood'].items():
            export_to_drive(
            image=img,
            description=f'2018_{stage}',
            subfolder='Flood',  
            parent_folder=parent_folder,
            scale=10  
            )
    
    print("\n所有测试导出任务已提交，请在 https://code.earthengine.google.com/tasks 查看任务进度")
    print(f"成功下载后，文件将保存在您的Google Drive的 {parent_folder} 文件夹中的子文件夹内")

    


In [47]:
test_download_fixed(2018, 2020, 'GEE')

开始测试下载 2018-2020 年的卫星数据...
获取Sentinel-2数据: 2018-2020
Sentinel-2 影像数量: 281
正在处理Sentinel-2影像...
Sentinel-2数据处理完成，共 281 景影像
创建year尺度的NDVI合成产品...
完成，共创建 3 个合成产品
开始导出年度NDVI示例...
导出任务已提交: NDVI_annual_2018 → GEE
导出任务已提交: NDVI_annual_2019 → GEE
导出任务已提交: NDVI_annual_2020 → GEE
提取2018年干扰事件数据...
提取2018年干旱和洪水事件数据...
获取Sentinel-2数据: 2018-2018
Sentinel-2 影像数量: 97
正在处理Sentinel-2影像...
Sentinel-2数据处理完成，共 97 景影像
获取Sentinel-2数据: 2017-2017
Sentinel-2 影像数量: 18
正在处理Sentinel-2影像...
Sentinel-2数据处理完成，共 18 景影像
获取Sentinel-2数据: 2018-2018
Sentinel-2 影像数量: 97
正在处理Sentinel-2影像...
Sentinel-2数据处理完成，共 97 景影像
处理干旱事件数据...
处理洪水事件数据...
警告: 洪水前期数据为空，使用空白影像
警告: 洪水期间数据为空，使用空白影像
干扰事件数据提取完成
导出干旱事件数据示例...
导出任务已提交: Drought_2018_before → GEE
导出任务已提交: Drought_2018_during → GEE
导出任务已提交: Drought_2018_after → GEE
导出洪水事件数据示例...
导出任务已提交: Flood_2018_before → GEE
导出任务已提交: Flood_2018_during → GEE
导出任务已提交: Flood_2018_after → GEE

所有测试导出任务已提交，请在 https://code.earthengine.google.com/tasks 查看任务进度
成功下载后，文件将保存在您的Google Drive的 GEE 文件夹中的子文件夹内
